In [ ]:
import SimpleITK as sitk
import nibabel as nib
import pandas as pd
import os
import numpy as np
from glob import glob
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

# Stack Perfusion

In [ ]:
## 분리되어 Co-Registration이 끝난 Perfusion 영상
srcDir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/target_img_seperate_registration'
destDir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/final_perfusion_img'
os.makedirs(destDir, exist_ok = True)

## 시간축 정보를 가져오기 위해, 분리되기 전 Perfusion 영상 불러오기(** 3차원 정보는 활용하지 않고 시간축 정보만 활용)
perfusion_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/original_data/target_img'

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcDir):
    if files:
        total += 1
        seq = os.path.basename(path)
        pt = path.split(os.path.sep)[-2] 
        print("# {} : merge img in pt: {} / seq : {}".format(total, pt, seq))

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcDir):
    if files:
        total += 1
        seq = os.path.basename(path)
        pt = path.split(os.path.sep)[-2] 
        print("# {} : merge img in pt: {} / seq : {}".format(total, pt, seq))
        
        img_list = []
        
        # first file header information - 분리 후, Co-Regi된 3차원 Perfusion영상의 위치정보를 가져옴(* Co-Regi되어 대표값 한개만 추출)
        reference_img_path = os.path.join(path, files[0])
        print("Reference file: {}".format(files[0]))
        
        reference_img = sitk.ReadImage(reference_img_path)
        reference_img_data = sitk.GetArrayFromImage(reference_img)
        
        print(reference_img.GetSize())
        print(np.array(reference_img.GetDirection()).reshape(3,3))
        print(reference_img.GetSpacing())
        print(reference_img.GetOrigin())
        
        reference_spacing = list(reference_img.GetSpacing())
        reference_origin = list(reference_img.GetOrigin())
        reference_direction = list(reference_img.GetDirection())     
        
        
        ### 3차원 파일을 시간축을 기준으로 쌓기
        check = 0
        for i in range(len(files)):
            check += 1
            target_file = glob(os.path.join(path, "{}_*".format(i+1)))
            print("%% {} : check {} slices ".format(check, i+1))
            
            if len(target_file) != 1:
                print("Somethins wrong....")
                continue
                
            print(os.path.basename(target_file[0]))
            
            img = sitk.ReadImage(target_file[0])
            img_data = sitk.GetArrayFromImage(img).astype('float32')
            
            print(img.GetSize())
            print(np.array(img.GetDirection()).reshape(3,3))
            print(img.GetSpacing())
            print(img.GetOrigin())
            print()
            
            img_list.append(sitk.GetImageFromArray(img_data, isVector = False))
         
    
        ## 하나의 이미지 파일로 변환
        MRI_img_final = sitk.JoinSeries(img_list)
        print(MRI_img_final.GetSize())
        
        # original 4D image - 헤더 정보 확인 후, 시간축 정보 불러와 입혀주기
        original_img_path = glob(os.path.join(perfusion_dir , pt, '*{}*'.format(seq)))
        print(original_img_path)
        
        if len(original_img_path) != 1: 
            print("Something wrong in original-perfusion...")
            continue
        
        print()
            
        original_perfusion = sitk.ReadImage(original_img_path[0])
        original_perfusion_data = sitk.GetArrayFromImage(original_perfusion)
        
        print("check original {}".format(seq))
        print(original_perfusion.GetSize())
        print(original_perfusion.GetSpacing())
        print(original_perfusion.GetOrigin())
        print(original_perfusion.GetDirection())
        print(np.array(original_perfusion.GetDirection()).reshape(4,4))
        print(original_perfusion.GetDimension)
        print()
        
        reference_spacing.append(list(original_perfusion.GetSpacing())[-1])
        reference_origin.append(list(original_perfusion.GetOrigin())[-1])
        
        for i in range(16):
            if (i % 4) == 3:
                reference_direction.insert(i, 0.0)
                #reference_direction.insert(i, np.array(original_perfusion.GetDirection)[i])
            elif(i//4) == 3:
                #reference_direction.inser(i, np.array(original_perfusion.GetDirection)[i])
                reference_direction.insert(i, 0.0)
        ### check the time dimension, 
        ### there is possibility that not always 0.0!!!
        
        
        # reference_direction[-1] = np.array(original_perfusion.GetDirection)[-1] --> 시간축 정보 삽입
        reference_direction[-1] = 1.0
        reference_direction = np.array(reference_direction)
        
        ## header insert - 처음 뽑아두었던 Co-Regi된 3차원 정보 삽입
        MRI_img_final.SetOrigin(reference_origin)
        MRI_img_final.SetDirection(reference_direction)
        MRI_img_final.SetSpacing(reference_spacing)
        
        ## final check
        print("check final {}".format(seq))
        print(MRI_img_final.GetSize())
        print(MRI_img_final.GetSpacing())
        print(MRI_img_final.GetDimension())
        print(MRI_img_final.GetOrigin())
        print(np.array(MRI_img_final.GetDirection()).reshape(4,4))
        
        ## set path and file name
        # new_file_name = "{}_{}_registered.nii.gz".format(pt, seq)
        new_file_name = "{}_{}_registered.nii".format(pt, seq)
        
#         pt_dir = os.path.join(destDir, pt)
#         os.makedirs(pt_dir, exist_ok = True)

        seq_dir = os.path.join(destDir, pt, seq)
        os.makedirs(seq_dir, exist_ok = True)
        
        final_path = os.path.join(seq_dir, new_file_name)
        
        if os.path.exists(final_path):
            print("File already exists!!!")
            continue
            
        sitk.WriteImage(MRI_img_final, final_path)
        
        
        print()